### Name: Aashis Khanal

In [178]:
import networkx as nx
import numpy as np
import itertools
import math
import json
import multiprocessing as mp

In [2]:
val_g1 = nx.read_edgelist('validation/validation_G1.edgelist', nodetype=int)
val_g2 = nx.read_edgelist('validation/validation_G2.edgelist', nodetype=int)

In [3]:
val_g1_A = nx.adj_matrix(val_g1).todense()
val_g2_A = nx.adj_matrix(val_g2).todense()

In [4]:
def load_mapping(file):
    mapping = {}
    for u, v in ([n.strip().split(' ') for n in open(file).readlines()]):
        mapping[int(u)] = int(v)
    return mapping

In [5]:
def knbrs(g=None, start=None, k=1):
    knbrs = list(nx.single_source_shortest_path_length(g, start, cutoff=k).keys())
    knbrs.remove(start)
    return set(knbrs)

In [47]:
def gm(costs):
    w = 1 / len(costs)
    _gm = 0.0
    for c in costs:
        _gm += w * math.log(c + 1)
    return math.exp(_gm)

In [6]:
val_mapping = load_mapping('validation/validation_seed_mapping.txt')

### Seed based

In [7]:
g1 = nx.read_edgelist('seedbased/G1.edgelist', nodetype=int)
g2 = nx.read_edgelist('seedbased/G2.edgelist', nodetype=int)

In [19]:
seed = load_mapping('seedbased/seed_node_pairs.txt')
seed_rev = dict([reversed(i) for i in seed.items()])
g1_seed = set(seed.keys())
g2_seed = set(seed.values())

In [35]:
def get_seed_nbrs(g, n, k, seed):
    nbrs = set(knbrs(g, n, k))
    return nbrs, len(nbrs), nbrs.intersection(seed)

In [173]:
def compute_mapping_score(i, itr_lim, ix, g1_len, m, g2_nodes, seed, g1_seed, g2_seed):
    print(f'[{i}/{itr_lim}]: Matching node from g1 {m}[{ix}/{g1_len}]')
    m = 1123
    g1_nbrs1, g1_nbrs1_len, g1_nbrs1_seed = get_seed_nbrs(g1, m, 1, g1_seed)
    g1_nbrs2, g1_nbrs2_len, g1_nbrs2_seed = get_seed_nbrs(g1, m, 2, g1_seed)
    g1_nbrs3, g1_nbrs3_len, g1_nbrs3_seed = get_seed_nbrs(g1, m, 3, g1_seed)
    sim = {}
    for jx, n in enumerate(g2_nodes, 0):
        print(f' -> to g2: {n} [{jx}/{g2_len}]', end='\r')
        
        c1, c2, c3 = 0, 0, 0
        m1, m2, m3 = 0, 0, 0
        g2_nbrs1, g2_nbrs1_len, g2_nbrs1_seed = get_seed_nbrs(g2, n, 1, g2_seed)
        
        for i, j in itertools.product(g1_nbrs1_seed, g2_nbrs1_seed):
            if seed[i]==j:
                m1 += 1
               
        c1 = m1 / (math.sqrt(g1_nbrs1_len) * math.sqrt(g2_nbrs1_len))
        
        if c1 > 0:
            g2_nbrs2, g2_nbrs2_len, g2_nbrs2_seed = get_seed_nbrs(g2, n, 2, g2_seed)
            
            for i, j in itertools.product(g1_nbrs2_seed, g2_nbrs2_seed):
                if seed[i] == j:
                    m2 += 1
            
            c2 =  (m1 * m2) / \
                (\
                 math.log(g1_nbrs1_len * g2_nbrs1_len) \
                 * \
                 math.sqrt(g1_nbrs2_len+1) * math.sqrt(g2_nbrs2_len)\
                )
        
        if c2 > 0:
            g2_nbrs3, g2_nbrs3_len, g2_nbrs3_seed = get_seed_nbrs(g2, n, 3, g2_seed)
            
            for i, j in itertools.product(g1_nbrs3_seed, g2_nbrs3_seed):
                if seed[i] == j:
                    m3 += 1

            c3 =  (m1 * m2 * m3) /\
                ( \
                 math.log(g1_nbrs1_len*g2_nbrs1_len*g1_nbrs2_len * g2_nbrs2_len+1) \
                 * \
                 math.sqrt(g1_nbrs3_len) * math.sqrt(g2_nbrs3_len)\
                )
            
            c = round(c1 + c2 + c3, 6)
            c1, c2, c3 = round(c1, 6), round(c2, 6), round(c3, 6)
            sim[(m, n)] = c
            print('  ->',[m, n], [m1, m2, m3], [c1, c2, c3], c, end='\r')
    
    if len(sim)>0:
        top = max(sim, key=sim.get)
        return top, sim[top]
    return None, None

In [174]:
MAP_PER_IT = 1000
MAPPING = {}.fromkeys(g1.nodes)
MAPPING.update(seed)
STRENGTH = {}.fromkeys(g1.nodes, 0.0)
for n in seed:
    STRENGTH[n] = float('inf')
ITR_LIM = (g1.number_of_nodes() - len(seed))//MAP_PER_IT + 1

In [ ]:
def make_params():
    

In [176]:
for i in range(ITR_LIM):
    g1_nodes = set([k for k in MAPPING if MAPPING[k] is None])
    g1_len = len(g1_nodes)
    g2_nodes = set(g2.nodes)-set(MAPPING.values())
    g2_len = len(g2_nodes)
    _mapping = {}.fromkeys(g1_nodes)
    _strength = {}.fromkeys(g1_nodes, 0.0)
    
    params = []
    for ix, m in enumerate(g1_nodes):
        params.append(i, ITR_LIM, ix, g1_len, m, g2_nodes, seed, g1_seed, g2_seed)
        n, s = compute_mapping_score(i, ITR_LIM, ix, g1_len, m, g2_nodes, 
                                            seed, g1_seed, g2_seed)
    for n, s in 
        if n is not None and strength > _strength[m]:
            _mapping[m] = n
            _strength[m] = s
            
            
    top_strength = sorted(_strength.items(), key=lambda kv:kv[1], reverse=True)\
                    [0:min(MAP_PER_IT, len(_strength))]
    
    for u, s in top_strength:
        if s>STRENGTH[u]:
            MAPPING[u] = _mapping[u]
            STRENGTH[u] = s
        
    if None not in node_mapping.values():
        break

In [39]:
def write_mappings_t0_file(file):
    mappings = [f'{a} {b}\n' for a, b in list(sim_mapping.items())]
    f = open(file, 'w')
    f.writelines(mappings)
    f.flush()

In [117]:
gm([0.144338, 0.01707, 0.307106])

1.1501080876154515

In [118]:
gm([0.063844, 0.041146, 2.177724])

1.5211378953575865

In [91]:
seed

{0: 3389,
 1: 2199,
 2: 269,
 3: 3398,
 4: 4338,
 5: 1268,
 6: 2373,
 7: 2023,
 8: 1371,
 9: 2674,
 10: 3467,
 11: 2561,
 12: 4343,
 13: 971,
 14: 264,
 15: 3474,
 16: 3366,
 17: 3689,
 18: 3871,
 19: 617,
 20: 4045,
 21: 1954,
 22: 76,
 23: 2689,
 24: 3516,
 25: 3722,
 26: 3700,
 27: 1745,
 28: 211,
 29: 3833,
 30: 1918,
 31: 2764,
 32: 3614,
 33: 2178,
 34: 3206,
 35: 1244,
 36: 2730,
 37: 1289,
 38: 1597,
 39: 844,
 40: 1877,
 41: 2263,
 42: 1591,
 43: 3019,
 44: 1031,
 45: 2960,
 46: 3502,
 47: 972,
 48: 267,
 49: 1445,
 50: 2237,
 51: 4456,
 52: 3688,
 53: 3291,
 54: 2940,
 55: 1035,
 56: 1407,
 57: 1814,
 58: 1274,
 59: 2241,
 60: 498,
 61: 4163,
 62: 2522,
 63: 1611,
 64: 1726,
 65: 332,
 66: 4026,
 67: 2236,
 68: 3838,
 69: 893,
 70: 4042,
 71: 1361,
 72: 1857,
 73: 126,
 74: 1621,
 75: 3845,
 76: 4460,
 77: 1129,
 78: 2469,
 79: 240,
 80: 3651,
 81: 995,
 82: 1378,
 83: 2998,
 84: 1606,
 85: 4319,
 86: 1971,
 87: 1500,
 88: 4046,
 89: 4284,
 90: 3181,
 91: 4352,
 92: 1791,
 93